# 🛠️ Zaawansowane użycie narzędzi z modelami GitHub (.NET)

## 📋 Cele nauki

Ten notebook pokazuje wzorce integracji narzędzi na poziomie korporacyjnym, wykorzystując Microsoft Agent Framework w .NET z modelami GitHub. Nauczysz się budować zaawansowane agentów z wieloma wyspecjalizowanymi narzędziami, korzystając z silnego typowania w C# oraz funkcji korporacyjnych .NET.

**Zaawansowane możliwości narzędzi, które opanujesz:**
- 🔧 **Architektura wielonarzędziowa**: Tworzenie agentów z wieloma wyspecjalizowanymi funkcjami
- 🎯 **Bezpieczne typowanie narzędzi**: Wykorzystanie walidacji w czasie kompilacji w C#
- 📊 **Wzorce narzędzi korporacyjnych**: Projektowanie narzędzi gotowych do produkcji i obsługa błędów
- 🔗 **Kompozycja narzędzi**: Łączenie narzędzi w złożone przepływy biznesowe

## 🎯 Korzyści z architektury narzędzi .NET

### Funkcje narzędzi korporacyjnych
- **Walidacja w czasie kompilacji**: Silne typowanie zapewnia poprawność parametrów narzędzi
- **Wstrzykiwanie zależności**: Integracja kontenera IoC do zarządzania narzędziami
- **Wzorce Async/Await**: Wykonywanie narzędzi bez blokowania z odpowiednim zarządzaniem zasobami
- **Strukturalne logowanie**: Wbudowana integracja logowania do monitorowania wykonania narzędzi

### Wzorce gotowe do produkcji
- **Obsługa wyjątków**: Kompleksowe zarządzanie błędami z typowanymi wyjątkami
- **Zarządzanie zasobami**: Właściwe wzorce usuwania i zarządzania pamięcią
- **Monitorowanie wydajności**: Wbudowane metryki i liczniki wydajności
- **Zarządzanie konfiguracją**: Typowana konfiguracja z walidacją

## 🔧 Architektura techniczna

### Główne komponenty narzędzi .NET
- **Microsoft.Extensions.AI**: Ujednolicony warstwa abstrakcji narzędzi
- **Microsoft.Agents.AI**: Orkiestracja narzędzi na poziomie korporacyjnym
- **Integracja z modelami GitHub**: Wydajny klient API z pulą połączeń

### Pipeline wykonania narzędzi
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Kategorie narzędzi i wzorce

### 1. **Narzędzia do przetwarzania danych**
- **Walidacja danych wejściowych**: Silne typowanie z adnotacjami danych
- **Operacje transformacji**: Typowane konwersje danych i formatowanie
- **Logika biznesowa**: Narzędzia do obliczeń i analiz specyficznych dla domeny
- **Formatowanie danych wyjściowych**: Generowanie odpowiedzi w ustrukturyzowanej formie

### 2. **Narzędzia integracyjne**
- **Konektory API**: Integracja usług RESTful z HttpClient
- **Narzędzia bazodanowe**: Integracja Entity Framework do dostępu do danych
- **Operacje na plikach**: Bezpieczne operacje na systemie plików z walidacją
- **Usługi zewnętrzne**: Wzorce integracji z usługami zewnętrznymi

### 3. **Narzędzia użytkowe**
- **Przetwarzanie tekstu**: Narzędzia do manipulacji i formatowania ciągów znaków
- **Operacje na datach/czasie**: Obliczenia dat/czasu uwzględniające kulturę
- **Narzędzia matematyczne**: Precyzyjne obliczenia i operacje statystyczne
- **Narzędzia walidacyjne**: Walidacja reguł biznesowych i weryfikacja danych

## ⚙️ Wymagania wstępne i konfiguracja

**Środowisko programistyczne:**
- .NET 9.0 SDK lub wyższy
- Visual Studio 2022 lub VS Code z rozszerzeniem C#
- Dostęp do API modeli GitHub

**Wymagane pakiety NuGet:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Konfiguracja środowiska (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Gotowy, aby budować agentów na poziomie korporacyjnym z potężnymi, typowanymi narzędziami w .NET? Zbudujmy profesjonalne rozwiązania! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego języku źródłowym powinien być uznawany za autorytatywne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
